# 1. Count labels in DB

In [ ]:
import os
import torch
from tqdm import tqdm
db_type = 'NoKGenc'
db_name = 'px'
size = 1000
SPLIT = ['train','valid','test']
labels = list()

for split in SPLIT:
    db_new = torch.load(f'/home/ssbae/bae/workspace/kgtxt/kg_txt_multimodal/gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
    for label in tqdm(db_new['label']):
        labels+=label
label_count = dict()
for label in tqdm(labels):
    if label not in label_count:
        label_count[label]=0
    label_count[label]+=1
label_count.pop(-100)
print(len(label_count))

# Prepare essential files
NUM_SPECIAL_TOKENS = 3
id2label = torch.load(f'/home/ssbae/bae/workspace/kgtxt/kg_txt_multimodal/gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
label2id = {v:k for k,v in id2label.items()}
# id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'/home/ssbae/bae/workspace/kgtxt/kg_txt_multimodal/gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
# label2entity = {k:id2entity[v] for k,v in label2id.items()}

In [ ]:
ori_label_count = label_count.copy()

In [ ]:
a = torch.tensor([1,2,3,4,5])
b = torch.tensor([1,2,3,4,5])
a/b

In [ ]:
import os
import torch
from tqdm import tqdm
db_type = 'UnifiedUniKGenc'
db_name = 'dx,prx'
size = 2000
id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/node2uninode')
id2label

## Visualize histogram

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

lower = 1000
upper = 1e10
NUM_SPECIAL_TOKENS = 3
db_name = 'px'

# Filter out labels
if db_name == 'px':
    triples = [x.split() for x in open(os.path.join(f'../gtx/data/{db_name}','train2id.txt')).read().splitlines()[1:]]
    drugname_nodes = set([int(t)+NUM_SPECIAL_TOKENS for h,t,r in triples if int(r) in [3,5,6]])
    label_count = {k:v for k,v in ori_label_count.items() if label2id[k] in drugname_nodes}
filtered_label_counts = {k:v for k,v in label_count.items() if (v>lower) and (v<upper)}
num_labels = np.array([count for count in filtered_label_counts.values()])
print(len(num_labels))

# Plot histogram of label distribution
plt.hist(num_labels, bins=np.arange(num_labels.min(), num_labels.max() + 1000, 1000), rwidth=0.7,
         density=True, cumulative=True,
         range=(num_labels.min(), num_labels.max()))
plt.title(f'l:{lower},u:{upper},#labels:{len(num_labels)}')
# plt.savefig(f'data/{db_name}/edlabel_distribution.png')
plt.show()

# Generate dictionary for label --> AdmPred labels
# ap_label = {v:k for k,v in enumerate(label_count)}
# torch.save(ap_label,f'data/{db_name}/label2edlabel')
# print([label2entity[k] for k in torch.load(f'data/{db_name}/label2edlabel').keys()])

In [ ]:
a = torch.eye(3)
b = torch.rand(3,4)
for _a,_b in zip(a,b):
    print(len(_a))
    print(_b)

# 2. Convert DB for AdmPred task

In [ ]:
import os
import torch
import numpy as np
from numpy.random import choice
from tqdm import tqdm
db_name = 'dx,prx'
size = 2000
DB_type = ['NoKGenc','UniKGenc','UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

# # Prepare essential files
# NUM_SPECIAL_TOKENS = 3
# id2label = torch.load(f'data/{db_name}_{size}/{db_name}_{db_type}/id2label')
# label2id = {v:k for k,v in id2label.items()}
# id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
# label2entity = {k:id2entity[v] for k,v in label2id.items()}

In [ ]:
adm2label = torch.load(f"../gtx/data/{'px' if db_name=='dx,prx' else 'dx,prx'}/adm2label")
label2aplabel = torch.load(f"../gtx/data/{'px' if db_name=='dx,prx' else 'dx,prx'}/label2aplabel")

for db_type in DB_type:
    # Prepare essential files
    if "Unified" not in db_type:
        NUM_SPECIAL_TOKENS = 3
        id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
        label2id = {v:k for k,v in id2label.items()}
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        label2entity = {k:id2entity[v] for k,v in label2id.items()}
    else:
        id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}

    if db_type == 'NoKGenc':
        global_label = list()
    idx = 0
    for split in SPLIT:
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        db_new = dict()
        for k in db:
            if k in ['label_mask','rc_index']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'NoKGenc':
                if id2entity[_input[1]] in adm2label:
                    if any([True if label in label2aplabel else False for label in adm2label[id2entity[_input[1]]]]):
                        for k in db_new:
                            if k not in ['label','label_mask','rc_index']:
                                db_new[k].append(db[k][in_db_idx])
                        aplabel = [label2aplabel[k] for k in adm2label[id2entity[_input[1]]] if k in label2aplabel]
                        db_new['label'].append(aplabel)
                        global_label.append(aplabel)
                    else:
                        global_label.append(None)
                else:
                    global_label.append(None)
            else:
                if global_label[idx] is not None:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(global_label[idx])
            idx += 1
        print(f'{db_name},{size},{split}',len(db_new['label']))
        os.makedirs(f'../gtx/data/adm/{db_name}_{size}/{db_name}_{db_type}/{split}')
        torch.save(db_new,f'../gtx/data/adm/{db_name}_{size}/{db_name}_{db_type}/{split}/db')

                            
                
                
            

In [ ]:
# Open Adm Pred DB
labels = list()
db_new = torch.load(f'data/adm/{db_name}_{size}/{db_name}_NoKGenc/train/db')
num_samples = len(db_new['label'])
for label in tqdm(db_new['label']):
    labels+=label

# Get sample frequency based class weights
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
aplabel2weight = compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(labels), 
                     y = labels)
torch.save(aplabel2weight,f'data/{db_name}/adm_class_weight')


# 3. Convert DB for ErrorDetection for Px,Dx

In [ ]:
# Load D_ICD files
import torch
import pandas as pd
d_proc = pd.read_csv("../gtx/data/mimic_data/D_ICD_PROCEDURES.csv")
d_diag = pd.read_csv("../gtx/data/mimic_data/D_ICD_DIAGNOSES.csv")

In [ ]:
def reformat(code, is_diag):
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

In [ ]:
d_diag['absolute_code'] = d_diag.apply(lambda row: str(reformat(str(row[1]), True)), axis=1)
d_proc['absolute_code'] = d_proc.apply(lambda row: str(reformat(str(row[1]), False)), axis=1)

In [ ]:
d_diag['code_name'] = d_diag.apply(lambda row: str(row[3]).lower().strip(), axis=1)
d_proc['code_name'] = d_proc.apply(lambda row: str(row[3]).lower().strip(), axis=1)

In [ ]:
d_diag_dict = d_diag[['absolute_code','code_name']].to_dict()
d_code2name = {d_diag_dict['absolute_code'][idx]:d_diag_dict['code_name'][idx] for idx in range(len(d_diag))}
d_name2codecat = {d_diag_dict['code_name'][idx]:dict([('large',d_diag_dict['absolute_code'][idx].split(".")[0]),('small',d_diag_dict['absolute_code'][idx].split(".")[-1] if len(d_diag_dict['absolute_code'][idx].split("."))>1 else "")])  for idx in range(len(d_diag))}
d_proc_dict = d_proc[['absolute_code','code_name']].to_dict()
p_code2name = {d_proc_dict['absolute_code'][idx]:d_proc_dict['code_name'][idx] for idx in range(len(d_proc))}
p_name2codecat = {d_proc_dict['code_name'][idx]:dict([('large',d_proc_dict['absolute_code'][idx].split(".")[0]),('small',d_proc_dict['absolute_code'][idx].split(".")[-1] if len(d_diag_dict['absolute_code'][idx].split("."))>1 else "")])  for idx in range(len(d_proc))}


In [ ]:
code2name = dict()
for d in [d_code2name, p_code2name]:
    for k,v in d.items():
        code2name[k.lower()] = f'{v}'

In [ ]:
name2codecat = dict()
for d in [d_name2codecat, p_name2codecat]:
    for k,v in d.items():
        v['large'] = v['large'].lower()
        name2codecat[f'{k}'] = v

In [ ]:
code2name_abs = dict()
for d in [d_code2name, p_code2name]:
    for k,v in d.items():
        code2name_abs[k.lower()] = f'{v}'
codebook = dict()
for k in code2name_abs:
    try:
        large, small = k.split(".")
    except:
        large, small = k, ""
    if large not in codebook:
        codebook[large] = list()
    codebook[large].append(small)
    

In [ ]:
import os
import torch
from copy import deepcopy
import numpy as np
from numpy.random import choice
from tqdm import tqdm
db_name = 'dx,prx'
size = 2000
DB_type = ['NoKGenc','UniKGenc','UnifiedNoKGenc','UnifiedUniKGenc']#,'UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [ ]:
db_type = 'UnifiedNoKGenc'
uniid2entity = {v:k.split('\t')[0].split('^^')[0].strip('"').replace('\\"','"') for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_UnifiedNoKGenc/unified_node').items()}
tot_amt = dict()
for split in SPLIT:
    db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
    for idx in range(len(db['input'])):
        ids = [x for x in db['input'][idx] if x>7]
        for _id in ids:
            if _id not in tot_amt:
                tot_amt[_id] = 0
            tot_amt[_id]+=1
code_frequency = {'.'.join([name2codecat[uniid2entity[k]]['large'],name2codecat[uniid2entity[k]]['small']]):1/v for k,v in tot_amt.items()}
uniid2entity

In [ ]:
def reformat(code, is_diag):
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('e'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

In [ ]:
def corrupt_input_and_generate_label(inputs, mode, id2entity, entity2id, code_frequency=None):
    if code_frequency is None:
        raise ValueError("Must turn on non-uniform sampling")
    try:
        inputs=np.array(inputs)
        inputs_ori = inputs.copy()
        input_entities = np.array([id2entity[x] if x in id2entity else x for x in inputs])

        if mode == 's':
            codes = list()
            f = list()
            for x in input_entities:
                if ("icd9_code" in x) and (x not in codes):
                    is_diag = True if 'diag' in x.split('/')[1] else False
                    code = name2codecat[code2name[reformat(x.split('/')[-1].strip(">"),is_diag = is_diag)]]
                    small_cat_length = len(codebook[code['large']])
                    if small_cat_length>=2:
                        codes.append(x)
                        f.append(code_frequency[".".join([code['large'],code['small']])])
            p = [x/sum(f) for x in f]
            corruption_target_codes = choice(codes,size=max(int(len(codes)*0.25),1), replace=False, p=p) 
            corruption_targets_idx = np.array([np.where(input_entities==code)[0][0] for code in corruption_target_codes])
            for corruption_target in corruption_targets_idx:
                code_entity = input_entities[corruption_target].split("/")
                header = '/'.join(code_entity[:-1])
                is_diag = True if 'diag' in header else False
                code = reformat(code_entity[-1].strip(">"),is_diag=is_diag)

                target_literal_idx = np.where(entity2id[code2name[code]] == inputs)[0][0]

                icd_code = code2name[code]
                codecat = name2codecat[icd_code]
                large_cat, small_cat = codecat['large'], codecat['small']
                small_cat_lists = codebook[large_cat].copy()

                small_cat_lists.remove(small_cat)
                for _ in range(50):
                    corrupted_small_cat = choice(small_cat_lists)
                    if code2name[".".join([large_cat,corrupted_small_cat])] in entity2id:
                        ERROR_FLAG=False
                        break
                    else:
                        ERROR_FLAG = True
                if ERROR_FLAG:
                    raise ValueError()
                inputs[target_literal_idx] = entity2id[code2name[".".join([large_cat,corrupted_small_cat])]]
                inputs[corruption_target] = entity2id['/'.join([header,"".join([large_cat,corrupted_small_cat])])+">"]
            labels = ~(inputs_ori==inputs)
    except:
        inputs = None
        labels = None
    return inputs, labels

In [ ]:
for db_type in DB_type:
    print(db_type)
    if db_type == 'NoKGenc':
        global_sample = list()
    idx = 0
    for split in SPLIT:
        # Prepare essential files
        NUM_SPECIAL_TOKENS = 3
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0].strip('"').replace('\\"','"') for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        id2entity[0]='PAD'
        id2entity[1]='MASK'
        id2entity[2]='CLS'
        entity2id = {v:k for k,v in id2entity.items()}
        uniid2entity = {v:k.split('\t')[0].split('^^')[0].strip('"').replace('\\"','"') for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_UnifiedNoKGenc/unified_node').items()}
        entity2uniid = {v:k for k,v in uniid2entity.items()}
        id2uniid = {k:entity2uniid[v] for k,v in id2entity.items() if v in entity2uniid}
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        db_new = dict()
        for k in db:
            if k in ['rc_index', 'label_mask']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'NoKGenc':
                x, y = corrupt_input_and_generate_label(inputs=db['input'][in_db_idx],mode='s',id2entity=id2entity, entity2id=entity2id, code_frequency=code_frequency)
                sample = {
                    'input': x,
                    'label': y,
                } if x is not None else None
                if sample is not None:
                    for k in db_new:
                        if k in sample:
                            db_new[k].append(sample[k].tolist())
                        else:
                            db_new[k].append(db[k][in_db_idx])
                global_sample.append(sample)
            else:
                if global_sample[idx] is not None:
                    sample = {k:global_sample[idx][k].copy() for k in global_sample[idx]}
                    actual_input = np.array(db['input'][in_db_idx].copy())
                    # Convert Non-unified sample to unifieid sample
                    if 'Unified' in db_type:
                        living_ids = sample['input'][sample['label']]
                        convertable_ids = np.array([True if living_id in id2uniid else False for living_id in living_ids])
                        sample['label'][sample['label']==True] = convertable_ids
                        actual_input[sample['label']] = np.array([id2uniid[x] for x in living_ids[convertable_ids]])
                    for k in db_new:
                        if k not in sample:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(sample['label'].tolist())
                    if 'Unified' in db_type:
                        db_new['input'].append(actual_input.tolist())
                    else:
                        db_new['input'].append(sample['input'].tolist())
                            
            idx += 1
        print('*'*50)
        print([uniid2entity[x] if 'Unified' in db_type else id2entity[x] for x in np.array(db['input'][0])[np.where(np.array(db_new['label'][0])==True)]])
        print([uniid2entity[x] if 'Unified' in db_type else id2entity[x] for x in np.array(db_new['input'][0])[np.where(np.array(db_new['label'][0])==True)]])
        print(np.where(np.array(db_new['label'][0])==True))
        print('-'*50)
        print(f'{db_name},{size},{split}',len(db_new['label']))
        print(list(db_new.keys()))
        print('*'*50)
        os.makedirs(f'../gtx/data/ed/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
        torch.save(db_new,f'../gtx/data/ed/{db_name}_{size}/{db_name}_{db_type}/{split}/db')            

# 3-1. Convert DB for ErrorDetection for Rx

In [3]:
import os
import torch
import pandas as pd
from copy import deepcopy
import numpy as np
from numpy.random import choice
from tqdm import tqdm
db_name = 'px'
size = 1000
DB_type = ['UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [4]:
# Prepare essential files
NUM_SPECIAL_TOKENS = 3
id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
id2entity[0]='PAD'
id2entity[1]='MASK'
id2entity[2]='CLS'
entity2id = {v:k for k,v in id2entity.items()}
uniid2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_UnifiedNoKGenc/unified_node').items()}
entity2uniid = {v:k for k,v in uniid2entity.items()}
uniid2id = {k:entity2id[v] for k,v in uniid2entity.items() if v in entity2id}

In [5]:
px = pd.read_csv('../gtx/data/mimic_data/PRESCRIPTIONS.csv')
px['DOSE'] = px.apply(lambda row: str(row[14])+str(row[15]), axis=1)
px_dict = px.to_dict()
CATEGORIES = ['DRUG_TYPE', 'DRUG', 'ROUTE', 'FORMULARY_DRUG_CD', 'DOSE']

/home/sjpark/miniconda3/envs/sjpark/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/sjpark/miniconda3/envs/sjpark/lib/python3.7/site-packages/pandas/core/dtypes/common.py:1776: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  npdtype = np.dtype(dtype)


In [6]:
pxcat2ids = {k:[f'"{str(x).lower()}"' for x in list(set(px_dict[k].values()))] for k in CATEGORIES}
id2pxcat = dict()
for k in pxcat2ids:
    pxcat2ids[k] = [entity2uniid[x] for x in pxcat2ids[k] if x in entity2uniid]
    for v in pxcat2ids[k]:
        id2pxcat[v]=k

In [7]:
db_type = 'UnifiedNoKGenc'
uniid2entity = {v:k.split('\t')[0].split('^^')[0].strip('"').replace('\\"','"') for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_UnifiedNoKGenc/unified_node').items()}
px2freq = {k:{v:0 for v in vl} for k,vl in pxcat2ids.items()}
# for k,vl in pxcat2ids.items()
for split in SPLIT:
    db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
    for idx in range(len(db['input'])):
        ids = [x for x in db['input'][idx] if x>7]
        for _id in ids:
            if _id in id2pxcat:
                px2freq[id2pxcat[_id]][_id]+=1

In [8]:
import torch
label = torch.tensor([[1,1,0,0],[0,0,0,1],[1,1,1,1]])
torch.where(label==1, dim=1)

TypeError: where() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (Tensor condition)
 * (Tensor condition, Tensor input, Tensor other)
 * (Tensor condition, Number self, Tensor other)
 * (Tensor condition, Tensor input, Number other)
 * (Tensor condition, Number self, Number other)


In [9]:
def corrupt_input_and_generate_label(inputs, literal_pos, pxcat2ids, id2pxcat):
#     try:
    corruptable_idx = np.array([i for i,(x,p) in enumerate(zip(inputs, literal_pos)) if (p==1) and (x in id2pxcat)])
    inputs=np.array(inputs)
    inputs_ori = inputs.copy()
    corruption_targets_idx = choice(corruptable_idx,max(int(len(corruptable_idx)*0.2),1))
    for corruption_target_idx in corruption_targets_idx:
        corruption_target_id = inputs[corruption_target_idx]
        pxcat = id2pxcat[corruption_target_id]
        candidate = np.array(pxcat2ids[pxcat])
        candidate[candidate!=corruption_target_id]
        corrupted_target_id = choice(candidate)
        inputs[corruption_target_idx] = corrupted_target_id

    labels = ~(inputs_ori==inputs)
#     except:
#         inputs = None
#         labels = None
    return inputs, labels

In [12]:
for db_type in DB_type:
    if db_type == 'UnifiedNoKGenc':
        global_sample = list()
    print(db_type)
    
    idx = 0
    for split in SPLIT:
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        db_new = dict()
        for k in db:
            if k in ['rc_index', 'label_mask']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'UnifiedNoKGenc':
                x, y = corrupt_input_and_generate_label(inputs=db['input'][in_db_idx],literal_pos=db['label_mask'][in_db_idx],pxcat2ids=pxcat2ids,id2pxcat=id2pxcat)
                sample = {
                    'input': x,
                    'label': y,
                } if x is not None else None
                if sample is not None:
                    for k in db_new:
                        if k in sample:
                            db_new[k].append(sample[k].tolist())
                        else:
                            db_new[k].append(db[k][in_db_idx])
                global_sample.append(sample)
            else:
                if global_sample[idx] is not None:
                    sample = {k:global_sample[idx][k].copy() for k in global_sample[idx]}
                    actual_input = np.array(db['input'][in_db_idx].copy())
                    # Convert Non-unified sample to unifieid sample
                    if 'Unified' not in db_type:
                        living_ids = sample['input'][sample['label']]
                        actual_input[sample['label']] = np.array([uniid2id[x] for x in living_ids])
                    for k in db_new:
                        if k not in sample:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(sample['label'].tolist())
                    if 'Unified' not in db_type:
                        db_new['input'].append(actual_input.tolist())
                    else:
                        db_new['input'].append(sample['input'].tolist())
                            
            idx += 1
        print('*'*50)
        print([uniid2entity[x] if 'Unified' in db_type else id2entity[x] for x in np.array(db['input'][0])[np.where(np.array(db_new['label'][0])==True)]])
        print([uniid2entity[x] if 'Unified' in db_type else id2entity[x] for x in np.array(db_new['input'][0])[np.where(np.array(db_new['label'][0])==True)]])
        print(np.where(np.array(db_new['label'][0])==True))
        print('-'*50)
        print(f'{db_name},{size},{split}',len(db_new['label']))
        print(list(db_new.keys()))
        print('*'*50)
        os.makedirs(f'../gtx/data/ed/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
        torch.save(db_new,f'../gtx/data/ed/{db_name}_{size}/{db_name}_{db_type}/{split}/db')               

UnifiedNoKGenc


23663it [03:21, 117.53it/s]


**************************************************
['dexa2', '325-650mg', 'po/ng', '100ml', 'folic acid', 'iv', 'main', '2g', 'po', 'main', 'levetiracetam', 'iv', '500mg', '100mg', 'phenytoin sodium', 'ns250', 'calcg2/100ns', '2-4g', 'po/ng', 'base', '100mg', '0.9% sodium chloride', 'infl0.5lf', 'im', '1000mg', 'phenytoin sodium', '1mg', 'base', 'iv', 'naclflush', 'hydralazine', '10mg', 'iv drip', 'nitroprusside sodium', 'ns100', 'base', '4mg', 'dexa4i', 'cefa10i', '1syr', 'syrg1', '2gm', 'base', 'main', '0.9% sodium chloride', '1-2bag']
['lido45i', '60-80meq', 'subcut', '167ml', 'ceftaroline', 'os', 'base', '7400unit', 'interspace', 'base', 'lotensin', 'periphnerve', '20-meq', '1mcg', '*nf*  loteprednol 0.2% ophth', 'phen10es', 'ambi5', '400ml', 'irr', 'main', '275mg', 'citric acid/sodium citrate', 'pseud30l', 'intrapericardial', '1-2loz', 'sodium citrate/citric acid', '6200unit', 'main', 'td', 'cloz25', 'flovent hfa', '0.124-0.5mg', 'po/ng', 'vitamin c', 'proc5l', 'main', '1310mg', '

1000it [00:08, 116.33it/s]


**************************************************
['main', 'base', 'potassium phosphate', 'iv', 'morphine sulfate', 'ns', 'ns250', 'main', 'morpconc', 'tp', '1appl', 'vancomycin', 'main', '2.5mg', 'olan5dtb', 'po', 'main', '0.25-0.5mg', 'kcl20/1000d51/2ns', '50ml', 'iso-osmotic dextrose', 'base', 'iv', 'asa81', 'd5 1/2ns', 'main', 'main', '5-10mg', 'infv', 'sw', 'sw50', 'iv', 'metrbase', 'chlorhexidine gluconate 0.12% oral rinse', 'oral', '1000ml', 'base', '0.25-0.5mg', 'furo40i', 'naclflush', 'iv', '250ml', 'lidocaine jelly 2% (urojet)', 'iv', 'cefazolin', 'iv', 'sw', '650mg', 'sc', 'po', 'iv', 'captopril', 'capt125', 'main', '6.25mg', 'levobase3', 'haloperidol', 'sw', 'levofloxacin', 'iv', 'oxycodone liquid', '0.5-1mg', 'oxyco5l', 'ipra2h', 'main', 'albu5500', '2gm', 'haloperidol', '25mg', 'metoprolol', 'iv', '20mg', 'potassium chl 20 meq / 1000 ml d5 1/2 ns', 'pantoprazole sodium', 'vancobase', 'famo20pm', 'iv', 'sw', 'ns', 'iv', '250ml', 'iv', 'sime80', 'furo40i', 'furosemide', 'i

...

**************************************************
['dexa2', '325-650mg', 'po/ng', '100ml', 'folic acid', 'iv', 'main', '2g', 'po', 'main', 'levetiracetam', 'iv', '500mg', '100mg', 'phenytoin sodium', 'ns250', 'calcg2/100ns', '2-4g', 'po/ng', 'base', '100mg', '0.9% sodium chloride', 'infl0.5lf', 'im', '1000mg', 'phenytoin sodium', '1mg', 'base', 'iv', 'naclflush', 'hydralazine', '10mg', 'iv drip', 'nitroprusside sodium', 'ns100', 'base', '4mg', 'dexa4i', 'cefa10i', '1syr', 'syrg1', '2gm', 'base', 'main', '0.9% sodium chloride', '1-2bag']
['lido45i', '60-80meq', 'subcut', '167ml', 'ceftaroline', 'os', 'base', '7400unit', 'interspace', 'base', 'lotensin', 'periphnerve', '20-meq', '1mcg', '*nf*  loteprednol 0.2% ophth', 'phen10es', 'ambi5', '400ml', 'irr', 'main', '275mg', 'citric acid/sodium citrate', 'pseud30l', 'intrapericardial', '1-2loz', 'sodium citrate/citric acid', '6200unit', 'main', 'td', 'cloz25', 'flovent hfa', '0.124-0.5mg', 'po/ng', 'vitamin c', 'proc5l', 'main', '1310mg', '

1000it [00:00, 13467.32it/s]


**************************************************
['main', 'base', 'potassium phosphate', 'iv', 'morphine sulfate', 'ns', 'ns250', 'main', 'morpconc', 'tp', '1appl', 'vancomycin', 'main', '2.5mg', 'olan5dtb', 'po', 'main', '0.25-0.5mg', 'kcl20/1000d51/2ns', '50ml', 'iso-osmotic dextrose', 'base', 'iv', 'asa81', 'd5 1/2ns', 'main', 'main', '5-10mg', 'infv', 'sw', 'sw50', 'iv', 'metrbase', 'chlorhexidine gluconate 0.12% oral rinse', 'oral', '1000ml', 'base', '0.25-0.5mg', 'furo40i', 'naclflush', 'iv', '250ml', 'lidocaine jelly 2% (urojet)', 'iv', 'cefazolin', 'iv', 'sw', '650mg', 'sc', 'po', 'iv', 'captopril', 'capt125', 'main', '6.25mg', 'levobase3', 'haloperidol', 'sw', 'levofloxacin', 'iv', 'oxycodone liquid', '0.5-1mg', 'oxyco5l', 'ipra2h', 'main', 'albu5500', '2gm', 'haloperidol', '25mg', 'metoprolol', 'iv', '20mg', 'potassium chl 20 meq / 1000 ml d5 1/2 ns', 'pantoprazole sodium', 'vancobase', 'famo20pm', 'iv', 'sw', 'ns', 'iv', '250ml', 'iv', 'sime80', 'furo40i', 'furosemide', 'i

1000it [00:00, 14314.05it/s]


**************************************************
['main', 'potassium chloride', 'main', 'dolasetron mesylate', 'lr', 'aten50', 'hydromorphone', 'magnesium sulfate', 'main', 'base', 'po', '1mg', 'sodium chloride 0.9%  flush', 'tp', '0.5-2mg', 'hydromorphone', 'iv', 'heparin flush cvl  (100 units/ml)', 'iv', 'artificial tear ointment', 'arti3.5o', 'miconazole powder 2%', '1appl', 'morp4-6', '40meq', '250ml', 'metrbase', 'iv', 'metronidazole', 'base', 'iv', 'sw', 'potassium chloride', 'pant40', '200mg', 'iv', 'base', '500ml', 'metoprolol', 'base', 'pantoprazole sodium', '40mg', 'ns500', '500ml', 'main']
['additive', 'saquinavir (invirase) cap', 'additive', 'humalog', 'albute', 'augm1.25l', 'cromolyn', 'methad', 'base', 'additive', 'po/ng', '500unit', 'fat emulsion 20%', 'td', '9900mg', 'rizatriptan', 'io', 'iron dextran', 'po/og', 'levetiracetam', 'quet50xr', 'lactic acid 12% lotion', '210ml', 'clon2p', '300-500unit', '11ml', 'reopi', 'periphnerve', 'flutamide', 'additive', 'po or enter

# 4. Modify Pretraining Input for KnowMix strategy

In [13]:
import os
import torch
from tqdm import tqdm, trange
db_name = 'px'
size = 1000
DB_type = ['UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [14]:
know_list = list()
for db_type in DB_type:
    global_idx=0
    # Prepare essential files
    if "Unified" not in db_type:
        NUM_SPECIAL_TOKENS = 3
        id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
        label2id = {v:k for k,v in id2label.items()}
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        label2entity = {k:id2entity[v] for k,v in label2id.items()}
    else:
        id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}
    for split in SPLIT:
        db = torch.load(f'../gtx/data/ed/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        db_new = {k:list() for k in db}
        db_new['knowledge'] = list()
        for idx in trange(len(db['input'])):
            for k in db:
                db_new[k].append(db[k][idx])
            if db_type == 'UnifiedNoKGenc':
                desc = [id2entity[x].replace('"','') if x>5 else "" for x in db_new['input'][idx]]
                know_list.append(desc)
                db_new['knowledge'].append(desc)
            else:
                db_new['knowledge'].append(know_list[global_idx])
            global_idx += 1
        os.makedirs(f'../gtx/data/ed/knowmix/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
        torch.save(db_new,f'../gtx/data/ed/knowmix/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
    id2desc = {k:v.split('^^')[0].replace('"','') if '^^' in v else "" for k,v in id2entity.items()}
    print(len(id2desc))
    torch.save(id2desc,f'../gtx/data/ed/knowmix/{db_name}_{size}/{db_name}_{db_type}/id2desc')
#     for k in db:
#         print(k,db[k][-1])
#     print("-"*30)
#     for k in db_new:
#         print(db_new[k][-1])
#     print("-"*30)
                            
                
                
            

100%|██████████| 1000/1000 [00:00<00:00, 9765.37it/s]


9169


100%|██████████| 1000/1000 [00:00<00:00, 481163.70it/s]


9169


In [ ]:
!pip install transformers

# 5. Convert DB for ReAdmPred task

In [ ]:
import os
import torch
import numpy as np
import pickle as pkl
from numpy.random import choice
from tqdm import tqdm
db_name = 'px'
size = 1000
DB_type = ['NoKGenc','UniKGenc','UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [ ]:
for db_type in DB_type:
    # Prepare essential files
    if "Unified" not in db_type:
        NUM_SPECIAL_TOKENS = 3
        id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
        label2id = {v:k for k,v in id2label.items()}
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        label2entity = {k:id2entity[v] for k,v in label2id.items()}
    else:
        id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}

    if db_type == 'NoKGenc':
        global_label = list()
    idx = 0
    for split in SPLIT:
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        readmDB_path = f"../gtx/data/temporal/readm_{'dxpx' if db_name=='dx,prx' else 'rx'}_{split}.pkl"
        with open(readmDB_path, 'rb') as f:
            print(readmDB_path)
            raw_readm_db = pkl.load(f)
        readm_db = dict()
        for sample in raw_readm_db:
            hadm_id, label = sample
            readm_db[str(hadm_id)] = label
        print("# samples: ",len(readm_db))
        db_new = dict()
        for k in db:
            if k in ['label_mask','rc_index']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'NoKGenc':
                hadm_id = id2entity[_input[1]].split('/')[-1].replace('>','')
                if hadm_id in readm_db:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(readm_db[hadm_id])
                    global_label.append(readm_db[hadm_id])
                else:
                    global_label.append(None)
            else:
                if global_label[idx] is not None:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(global_label[idx])
            idx += 1
        print(f'{db_name},{size},{split}',len(db_new['label']))
        if "Unified" in db_type:
            os.makedirs(f'../gtx/data/temporal/readm/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
            torch.save(db_new,f'../gtx/data/temporal/readm/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
            print("*** Saved! ***")
        print(f"100th sample", [id2entity[x] for x in db_new['input'][99][1:30]])
        print(f"100th label", db_new['label'][99])

# 6. Convert DB for NextDxPxPred task

In [ ]:
import os
import torch
import numpy as np
import pickle as pkl
from numpy.random import choice
from tqdm import tqdm
db_name = 'px'
size = 1000
DB_type = ['NoKGenc','UniKGenc','UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [ ]:
for db_type in DB_type:
    # Prepare essential files
    if "Unified" not in db_type:
        NUM_SPECIAL_TOKENS = 3
        id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
        label2id = {v:k for k,v in id2label.items()}
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        label2entity = {k:id2entity[v] for k,v in label2id.items()}
    else:
        id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}

    if db_type == 'NoKGenc':
        global_label = list()
    idx = 0
    for split in SPLIT:
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        nextdxDB_path = f"../gtx/data/temporal/nextdx_{'dxpx' if db_name=='dx,prx' else 'rx'}_{split}.pkl"
        with open(nextdxDB_path, 'rb') as f:
            print(nextdxDB_path)
            raw_nextdx_db = pkl.load(f)
        nextdx_db = dict()
        for sample in raw_nextdx_db:
            hadm_id, label = str(int(sample[0])), sample[1:]
            nextdx_db[hadm_id] = label
        print("# samples: ",len(nextdx_db))
        db_new = dict()
        for k in db:
            if k in ['label_mask','rc_index']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'NoKGenc':
                hadm_id = id2entity[_input[1]].split('/')[-1].replace('>','')
                if hadm_id in nextdx_db:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(nextdx_db[hadm_id])
                    global_label.append(nextdx_db[hadm_id])
                else:
                    global_label.append(None)
            else:
                if global_label[idx] is not None:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(global_label[idx])
            idx += 1
        print(f'{db_name},{size},{split}',len(db_new['label']))
        if "Unified" in db_type:
            os.makedirs(f'../gtx/data/temporal/nextdx/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
            torch.save(db_new,f'../gtx/data/temporal/nextdx/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
            print("*** Saved! ***")
        print(f"100th sample", [id2entity[x] for x in db_new['input'][99][100:105]])
        print(f"100th label", db_new['label'][99][:5])

# 7. Convert DB for Mortaility Pred task

In [ ]:
import os
import torch
import numpy as np
import pickle as pkl
from numpy.random import choice
from tqdm import tqdm
db_name = 'px'
size = 1000
# db_name = 'dx,prx'
# size = 2000
DB_type = ['NoKGenc','UniKGenc','UnifiedNoKGenc','UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [ ]:
for db_type in DB_type:
    # Prepare essential files
    if "Unified" not in db_type:
        NUM_SPECIAL_TOKENS = 3
        id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
        label2id = {v:k for k,v in id2label.items()}
        id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
        label2entity = {k:id2entity[v] for k,v in label2id.items()}
    else:
        id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}

    if db_type == 'NoKGenc':
        global_label = list()
    idx = 0
    for split in SPLIT:
        db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
        death180DB_path = f"../gtx/data/temporal/death180_{'dxpx' if db_name=='dx,prx' else 'rx'}_{split}.pkl"
        with open(death180DB_path, 'rb') as f:
            print(death180DB_path)
            raw_death180_db = pkl.load(f)
        death180_db = dict()
        for sample in raw_death180_db:
            hadm_id, label = sample
            death180_db[str(hadm_id)] = label
        print("# samples: ",len(death180_db))
        db_new = dict()
        for k in db:
            if k in ['label_mask','rc_index']:
                continue
            db_new[k] = list()
        for in_db_idx, _input in tqdm(enumerate(db['input'])):
            if db_type == 'NoKGenc':
                hadm_id = id2entity[_input[1]].split('/')[-1].replace('>','')
                if hadm_id in death180_db:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(death180_db[hadm_id])
                    global_label.append(death180_db[hadm_id])
                else:
                    global_label.append(None)
            else:
                if global_label[idx] is not None:
                    for k in db_new:
                        if k not in ['label','label_mask','rc_index']:
                            db_new[k].append(db[k][in_db_idx])
                    db_new['label'].append(global_label[idx])
            idx += 1
        print(f'{db_name},{size},{split}',len(db_new['label']))
        if "Unified" in db_type:
            os.makedirs(f'../gtx/data/temporal/death180/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
            torch.save(db_new,f'../gtx/data/temporal/death180/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
            print("*** Saved! ***")
        print(f"100th sample", [id2entity[x] for x in db_new['input'][99][1:5]])
        print(f"100th label", db_new['label'][99])

# 8. Prepare literal to word dictionary for word initialization

In [ ]:
import os
import torch
from tqdm import tqdm, trange
db_name = 'px'
size = 1000
DB_type = ['UnifiedNoKGenc','UnifiedUniKGenc']

In [ ]:
for db_type in DB_type:
    global_idx=0
    # Prepare essential files
    id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}
    desc = [id2entity[x].replace('"','') if x>5 else "" for x in id2entity]
    print(len(id2entity))
    print(id2entity)
    torch.save(desc,f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/lit2word')
    torch.save(desc,f'../gtx/data/knowmix/{db_name}_{size}/{db_name}_{db_type}/lit2word')

                            
                
                
            

# IMPORTANT. FIX CLS ATTN MASK

In [8]:
import os
import torch
import gc
from tqdm import tqdm, trange
db_name = 'dx,prx'
size = 2000
TASK = ["","adm/","ed/","readm/","Death30/"]
DB_type = ['UnifiedUniKGenc']
SPLIT = ['train','valid','test']

In [11]:
for fusion in [True,False]:
    for task in TASK:
        if fusion:
            task = task+'knowmix/'
        for db_type in DB_type:
            for split in SPLIT:
                try:
                    db = torch.load(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/{split}/db')
                except:
                    continue
                db_length = len(db['input'])
                for idx in trange(db_length):
                    _input, _mask = db['input'][idx], db['mask'][idx]
                    new_cls_mask = [1 if x!=0 else 0 for x in _input]
                    _mask[0] = torch.tensor(new_cls_mask)
                    db['mask'][idx] = _mask
                    if sum(db['mask'][idx][0])!= sum([x!=0 for x in _input]):
                        raise ValueError()
                os.makedirs(f'../gtx/fixed_data/{task}/{db_name}_{size}/{db_name}_{db_type}/{split}', exist_ok=True)
                torch.save(db,f'../gtx/fixed_data/{task}/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
                os.remove(f'../gtx/data/{task}/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
                del db
                gc.collect()

100%|██████████| 983/983 [00:02<00:00, 344.55it/s]


In [10]:
for fusion in [True,False]:
    for task in TASK:
        if fusion:
            task = task+'knowmix/'
        for db_type in DB_type:
            for split in SPLIT:
                path = f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/{split}/db'
                print(path)
                try:
                    db = torch.load(path)
                    
                except:
#                     print("fail")
                    continue
                
                db_length = len(db['input'])
                print(db['mask'][0][0])

../gtx/data/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/train/db
../gtx/data/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/valid/db
../gtx/data/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/test/db
../gtx/data/adm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/train/db
../gtx/data/adm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/valid/db
../gtx/data/adm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/test/db
../gtx/data/ed/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/train/db
../gtx/data/ed/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/valid/db
../gtx/data/ed/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/test/db
../gtx/data/readm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/train/db
../gtx/data/readm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/valid/db
../gtx/data/readm/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/test/db
../gtx/data/Death30/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/train/db
../gtx/data/Death30/knowmix/dx,prx_2000/dx,prx_UnifiedUniKGenc/valid/db
../gtx/data/Death30/knowmix/dx,prx_2000/dx,prx_Unifie

In [6]:
import shutil

for fusion in [True,False]:
    for task in TASK:
        if fusion:
            task = task+'knowmix/'
        for db_type in DB_type:
            count = 0
            try:
                shutil.copy(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/lit2word',
                           f'../gtx/fixed_data/{task}{db_name}_{size}/{db_name}_{db_type}/lit2word')
                count += 1
            except:
                pass
            try:
                shutil.copy(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/id2label',
                           f'../gtx/fixed_data/{task}{db_name}_{size}/{db_name}_{db_type}/id2label')
                count += 1
            except:
                pass
            try:
                shutil.copy(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/node2uninode',
                           f'../gtx/fixed_data/{task}{db_name}_{size}/{db_name}_{db_type}/node2uninode')
                count += 1
            except:
                pass
            try:
                shutil.copy(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/unified_node',
                           f'../gtx/fixed_data/{task}{db_name}_{size}/{db_name}_{db_type}/unified_node')
                count += 1
            except:
                pass
            try:
                shutil.copy(f'../gtx/data/{task}{db_name}_{size}/{db_name}_{db_type}/id2desc',
                           f'../gtx/fixed_data/{task}{db_name}_{size}/{db_name}_{db_type}/id2desc')
                count += 1
            except:
                pass
            print(task, db_type, count)

knowmix/ UnifiedUniKGenc 2
adm/knowmix/ UnifiedUniKGenc 1
ed/knowmix/ UnifiedUniKGenc 1
readm/knowmix/ UnifiedUniKGenc 1
Death30/knowmix/ UnifiedUniKGenc 1
 UnifiedUniKGenc 4
adm/ UnifiedUniKGenc 0
ed/ UnifiedUniKGenc 0
readm/ UnifiedUniKGenc 0
Death30/ UnifiedUniKGenc 0


# DB check

In [ ]:
import torch
import os
db_name = 'px'
size = 1000
db_type = 'UnifiedUniKGenc'
split = 'valid'
db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
if "Unified" not in db_type:
    NUM_SPECIAL_TOKENS = 3
    id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
    label2id = {v:k for k,v in id2label.items()}
    id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
    label2entity = {k:id2entity[v] for k,v in label2id.items()}
    entity2id = {v:k for k,v in id2entity.items()}
else:
    id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}
    entity2id = {v:k for k,v in id2entity.items()}

In [ ]:
print(id2entity)

# Extract HADM_ID

In [ ]:
import torch
import os
db_name = 'dx,prx'
size = 2000
db_type = 'NoKGenc'
if "Unified" not in db_type:
    NUM_SPECIAL_TOKENS = 3
    id2label = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/id2label')
    label2id = {v:k for k,v in id2label.items()}
    id2entity = {int(line.split('\t')[1])+NUM_SPECIAL_TOKENS:line.split('\t')[0].split('^^')[0] for line in open(os.path.join(f'../gtx/data/{db_name}','entity2id.txt')).read().splitlines()[1:]}
    label2entity = {k:id2entity[v] for k,v in label2id.items()}
    entity2id = {v:k for k,v in id2entity.items()}
else:
    id2entity = {v:k.split('\t')[0].split('^^')[0] for k,v in torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/unified_node').items()}
    entity2id = {v:k for k,v in id2entity.items()}

In [ ]:
import gc
seen_ids = list()
for split in ['train','valid','test']:
    db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')
    count = 0
    with open(f'{db_name}_{split}.txt','w') as f:
        for sample in db['input']:
            aid = id2entity[sample[1]].split('/')[-1].replace('>','')
            if aid in seen_ids and split != "train":
                print("oh my god!")
                print(aid)
                count +=1
            seen_ids.append(aid)
            f.write(aid+'\n')
    print(count)
    del db
    gc.collect()

In [ ]:
print(df_discharge_smmary[df_discharge_smmary['HADM_ID']==167118].values[0,1])

# Playground!

In [ ]:
import pickle as pkl

with open('../gtx/data/temporal/icd2int.pkl', 'rb') as f:
    icd2int = pkl.load(f)
    
with open('../gtx/data/temporal/nextdx_dxpx_test.pkl', 'rb') as f:
    nextdx_dxpx = pkl.load(f)
    
with open('../gtx/data/temporal/nextdx_rx_test.pkl', 'rb') as f:
    nextdx_rx = pkl.load(f)

with open('../gtx/data/temporal/readm_rx_test.pkl', 'rb') as f:
    readm_rx = pkl.load(f)
    
with open('../gtx/data/temporal/readm_dxpx_test.pkl', 'rb') as f:
    readm_dxpx = pkl.load(f)

In [ ]:
for sample in nextdx_dxpx:
    hadm_id, label = sample[0], sample[1:]
print(str(int(hadm_id)))
print(label)

In [ ]:
import torch
db = torch.load(f'../gtx/data/dx,prx_2000/dx,prx_NoKGenc/train/db')

In [ ]:
import pandas
df_diag = pandas.read_csv("../gtx/data/DIAGNOSES_ICD.csv")

In [ ]:
df_note = pandas.read_csv("../gtx/data/DIAGNOSES_ICD.csv")

In [25]:
import torch
db_name = 'dx,prx'
size = 2000
db_type = 'UnifiedNoKGenc'
split="test"
db = torch.load(f'../gtx/data/{db_name}_{size}/{db_name}_{db_type}/{split}/db')

In [26]:
_input, _mask = db['input'][0], db['mask'][0]
new_cls_mask = [1 if x!=0 else 0 for x in _input]
_mask[0] = torch.tensor(new_cls_mask)
db['mask'][0] = _mask

KeyError: 'mask'

In [19]:
db['mask'][0][0].sum()

tensor(77.)

In [20]:
sum([x!=0 for x in db['input'][0]])

77